Transformed Sectiontion Method
- "Its core principle involves conceptually converting the multi-material cross-section of a layered beam into an equivalent, fictitious cross-section composed entirely of a single, homogeneous material."
- "The transformation is achieved by adjusting the width of each material by a factor equal to its modular ratio (n_i = E_i / E_ref), relative to the chosen base material."

(hand hardness, grain form) -> density
density -> Youngs Modulus
(Young's Modulus, thickness, orientation) -> Bending stiffness

In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from snowpylot import caaml_parser


## Outline Transformed Section Method

**Layer Properties**

    rho_i: density of layer i, kg/m^3
    b_i=1: width of layer i, m (start with 1m as baseline)
    h_i: # thickness of layer i

**Calculation**

1. Calculate density from hand hardness and grain form

2. Calculate Young's Modulus from density

    **rho_0** = 917 # kg/m^3 (density of ice), from "A Closed form model..."

    **E_i**= 6.5e3 *(rho_i)**4.4 # From Gerling et al. (2017)

3. Select Reference Material in Slab (highest E)

    **E_ref** = max(E_i) # Choose material with highest E

4. Calculate modular ratio for each layer

    **n_i** = E_i / E_ref

5. Calculate transformed width of each layer

    **b_i_prime** = b_i * n_i # m (transformed width of layer i)

6. Calculate transformed area of each layer

    **A_i_prime** = b_i_prime * h_i # m^2 (transformed area of layer i)

7.  Locate Centroid of Layer

    **y_i** = sum of thickness of layers below i + h_i/2

8.  Locate Centroid of Transformed slab

    **Y_bar** = sum(A_i_prime * y_i) / sum(A_i_prime) # m (centroid of transformed slab)

9.  Calculate Moment of Inertia (I) of the Transformed Slab

    **I_i** = (1/12) * b_i_prime * h_i**3 # m^4 (moment of inertia of layer i)

    **d_i** = abs(y_i-Y_bar) # m (distance from centroid of layer i to centroid of transformed slab)

    **I_bar** = sum(I_i + A_i_prime * d_i**2) # m^4 (moment of inertia of transformed slab) NOTE: CHECK THIS EQUATION

10. Calculate Equivalent Bending Stiffness (D11) of the Transformed Slab

    **D11** = E_ref * I_bar # N*m^2 (bending stiffness of transformed slab)


parse_pits function: Parses all pits in a specified folder

In [ ]:
def parse_pits(folder_path):
    """
    Function to parse CAAML files in the specified folder
    """

    files = [
        f for f in os.listdir(folder_path) if f.endswith(".xml")
    ]  # List of all .xml files in the folder

    pits_list = []

    for file in files:  # iterate through each file in the folder
        file_path = folder_path + "/" + file  # create the file path
        pit = caaml_parser(file_path)  # parse the file
        pits_list.append(pit)

    return pits_list


Specify folders for 2020-2024 Water Years and parse files

In [3]:
# Define folders and parse pits

pits_19_20 = parse_pits("../snowpits/by_season/2019-2020")
pits_20_21 = parse_pits("../snowpits/by_season/2020-2021")
pits_21_22 = parse_pits("../snowpits/by_season/2021-2022")
pits_22_23 = parse_pits("../snowpits/by_season/2022-2023")
pits_23_24 = parse_pits("../snowpits/by_season/2023-2024")

all_pits = (
    pits_19_20 + pits_20_21 + pits_21_22 + pits_22_23 + pits_23_24
)  # list of all pits


Geldsetzer table of density from hand hardness and grain form

In [4]:
geldsetzer_df = pd.read_csv('geldsetzer_table.csv', index_col=0)

def get_density(hand_hardness, grain_form, df=geldsetzer_df):
    """
    Get density value for a specific hand hardness and grain form combination.
    
    Parameters:
    df (pd.DataFrame): The Geldsetzer table DataFrame
    hand_hardness (str): Hand hardness value (e.g., 'F-', '4F+', 'P-', etc.)
    grain_form (str): Grain form (e.g., 'PP', 'DF', 'RG', 'FC', 'DH', etc.)
    
    Returns:
    float: Density value, or NaN if not available
    """
    try:
        return df.loc[hand_hardness, grain_form]
    except KeyError:
        return np.nan

def convert_grain_form(layer):
    """
    Convert grain form to code needed for Geldsetzer table.
    
    Parameters:
    layer: Snow layer object with grain_form_primary attribute
    
    Returns:
    str: Grain form code for Geldsetzer table lookup
    """
    if layer.grain_form_primary.sub_grain_class_code in ["PPgp","RGmx","FCmx"]:
        return layer.grain_form_primary.sub_grain_class_code
    else:
        return layer.grain_form_primary.basic_grain_class_code


In [5]:
def calculate_weight_above_layer(layers, target_layer):
    """
    Calculate cumulative weight above a specified target layer (excludes target layer).

    Parameters:
    layers: List of snow layer objects
    target_layer: The layer to stop calculation at (weight calculated above this layer)
    
    Returns:
    float: Weight above target layer in kg/m², or NaN if calculation fails
    """
    weight_above = 0
    target_found = False
    
    for layer in layers:
        # Check if we've reached the target layer - if so, stop BEFORE including it
        if layer is target_layer:
            target_found = True
            break
            
        # Convert grain form to code needed for Geldsetzer table
        grain_form = convert_grain_form(layer)
        
        # Get density
        density = get_density(layer.hardness, grain_form)
        
        if pd.isna(density):
            return np.nan
            
        # Calculate weight
        thickness_m = (layer.thickness[0])/100  # convert cm to m
        weight = thickness_m * density  # Weight in kg (assuming 1 m^2 cross-section)
        weight_above += weight
    
    # If target layer was not found, return NaN
    if not target_found:
        return np.nan
        
    return weight_above


In [ ]:
def calculate_bending_stiffness(layers, target_layer):
    """
    Calculate bending stiffness above a specified target layer.
    
    Parameters:
    layers: List of snow layer objects
    target_layer: The layer to stop calculation at (bending stiffness calculated for slab above this layer)
    """

    return


In [ ]:

def filter_pits_with_hardness_and_grain_form(pits):
    """
    Filter pits to only include those with complete hardness and grain form data.
    
    Parameters:
    pits: List of pit objects
    
    Returns:
    list: Filtered list of pits with complete data
    """
    filtered_pits = []
    
    for pit in pits:
        all_layers_info = True
        layers = pit.snow_profile.layers
        for layer in layers:
            if layer.hardness is None or layer.grain_form_primary is None:
                all_layers_info = False
                break
        if all_layers_info:
            filtered_pits.append(pit)
            
    return filtered_pits

def filter_pits_with_layer_of_concern(pits):
    """
    Filter pits to only include those with at least one layer of concern.
    
    Parameters:
    pits: List of pit objects
    
    Returns:
    list: Filtered list of pits with layer of concern
    """
    filtered_pits = []
    
    for pit in pits:
        layers = pit.snow_profile.layers
        has_loc = any(layer.layer_of_concern for layer in layers)
        if has_loc:
            filtered_pits.append(pit)
            
    return filtered_pits

def find_layer_of_concern(pit):
    """
    Find the layer of concern in a pit.
    
    Parameters:
    pit: A single pit object
    
    Returns:
    layer: The layer of concern, or None if not found
    """
    layers = pit.snow_profile.layers
    
    # Find the layer of concern
    for layer in layers:
        if layer.layer_of_concern:
            return layer
            
    return None

def print_weight_statistics(df, weight_column, description):
    """
    Print summary statistics for weight data.
    
    Parameters:
    df: DataFrame containing weight data
    weight_column: Name of the weight column
    description: Description for the output
    """
    non_nan_results = df[df[weight_column].notna()]
    print(f"num pits with {description}: {len(non_nan_results)}")
    print(f"mean {description} (kg): {non_nan_results[weight_column].mean()}")
    print(f"median {description} (kg): {non_nan_results[weight_column].median()}")
    print()

def find_ECTP_propagation_layer(pit):
    """
    Find the ECTP propagation layer in a pit.
    
    Parameters:
    pit: A single pit object
    
    Returns:
    layer: The ECTP propagation layer, or None if not found
    """
    ECTs_list = pit.stability_tests.ECT  # list of ECTs for pit
    layers = pit.snow_profile.layers  # List of layers in pit
    
    for ECT in ECTs_list:  # For each ECT in pit
        if ECT.propagation:  # If ECTP
            prop_layer_depth_top = ECT.depth_top  # depth of top of propagation layer 
            
            # Find the layer of propagation in layers
            for layer in layers:
                if layer.depth_top == prop_layer_depth_top:
                    return layer
                    
    return None

def filter_pits_by_location(pits, location_type):
    """
    Filter pits by avalanche location type (crown or flank).
    
    Parameters:
    pits: List of pit objects
    location_type: String, either "crown" or "flank"
    
    Returns:
    list: Filtered list of pits at specified location
    """
    filtered_pits = []
    
    for pit in pits:
        if pit.core_info.location.pit_near_avalanche_location == location_type:
            filtered_pits.append(pit)
            
    return filtered_pits

def prepare_plotting_data(dataframes_and_labels):
    """
    Prepare multiple dataframes for plotting by combining them with labels.
    
    Parameters:
    dataframes_and_labels: List of tuples, each containing (dataframe, weight_column, label)
    
    Returns:
    pd.DataFrame: Combined dataframe ready for plotting
    """
    combined_data = []
    
    for df, weight_column, label in dataframes_and_labels:
        clean_df = df[df[weight_column].notna()].copy()
        clean_df['dataset'] = label
        clean_df['weight'] = clean_df[weight_column]
        combined_data.append(clean_df[['dataset', 'weight']])
    
    return pd.concat(combined_data, ignore_index=True)


## Find Pits with Hand Hardness and Grain Form



In [ ]:
# Find pits with hardness and grain form info
pits_with_hardness_and_grain_form = filter_pits_with_hardness_and_grain_form(all_pits)

print("Num pits with hardness and grain form info: ", len(pits_with_hardness_and_grain_form))


In [ ]:
# Find pits with indicated layer of concern
pits_with_layer_of_concern = filter_pits_with_layer_of_concern(pits_with_hardness_and_grain_form)

print("Num pits: ", len(all_pits))
print("Num pits with hand hardness and grain form, and layer of concern: ", len(pits_with_layer_of_concern))


In [ ]:
LOC_results = calculate_weights_for_layer_of_concern(pits_with_layer_of_concern)
LOC_results_df = pd.DataFrame(LOC_results)


In [ ]:
print_weight_statistics(LOC_results_df, 'weight_above_layer_of_concern', 'weight above layer of concern')


### Weight Above ECTP Layer of Propagation

In [ ]:
ECTP_results = calculate_weights_for_ECTP_propagation(pits_with_hardness_and_grain_form)
ECTP_results_df = pd.DataFrame(ECTP_results)


In [ ]:
print_weight_statistics(ECTP_results_df, 'weight_above_ECTP_layer_of_propagation', 'weight above ECTP layer of propagation')


## Layer of Concern for Pits on Crowns and Flanks

In [ ]:
pits_on_crowns = filter_pits_by_location(pits_with_hardness_and_grain_form, "crown")
pits_on_flanks = filter_pits_by_location(pits_with_hardness_and_grain_form, "flank")

print("num pits on crowns (with hand hardness and grain form): ", len(pits_on_crowns))
print("num pits on flanks (with hand hardness and grain form): ", len(pits_on_flanks))
    

In [ ]:
# Pits on crowns with LOC
pits_on_crowns_with_LOC = filter_pits_with_layer_of_concern(pits_on_crowns)

# Pits on flanks with LOC
pits_on_flanks_with_LOC = filter_pits_with_layer_of_concern(pits_on_flanks)

print("num pits on crowns (with LOC): ", len(pits_on_crowns_with_LOC))
print("num pits on flanks (with LOC): ", len(pits_on_flanks_with_LOC))


In [ ]:
# Calculate weight above LOC for pits on crowns and flanks

# Pits on crowns with LOC
pits_on_crowns_with_LOC_results = calculate_weights_for_layer_of_concern(pits_on_crowns_with_LOC)
pits_on_crowns_with_LOC_results_df = pd.DataFrame(pits_on_crowns_with_LOC_results)

# Pits on flanks with LOC
pits_on_flanks_with_LOC_results = calculate_weights_for_layer_of_concern(pits_on_flanks_with_LOC)
pits_on_flanks_with_LOC_results_df = pd.DataFrame(pits_on_flanks_with_LOC_results)


In [ ]:
# Summary Stats for pits_on_crowns_with_LOC_results
print("Crown pits:")
print_weight_statistics(pits_on_crowns_with_LOC_results_df, 'weight_above_layer_of_concern', 'weight above layer of concern')

# Summary Stats for pits_on_flanks_with_LOC_results  
print("Flank pits:")
print_weight_statistics(pits_on_flanks_with_LOC_results_df, 'weight_above_layer_of_concern', 'weight above layer of concern')


# Plot Results

In [ ]:
# Create side-by-side violin plots for LOC, ECTP, Crown, and Flank results

# Prepare the data for plotting using the helper function
data_for_plotting = [
    (LOC_results_df, 'weight_above_layer_of_concern', 'Indicated Layer of Concern (all pits)'),
    (ECTP_results_df, 'weight_above_ECTP_layer_of_propagation', 'ECTP Layer of Propagation'),
    (pits_on_crowns_with_LOC_results_df, 'weight_above_layer_of_concern', 'Indicated Layer of Concern (Pits on Crowns)'),
    (pits_on_flanks_with_LOC_results_df, 'weight_above_layer_of_concern', 'Indicated Layer of Concern (Pits on Flanks)')
]

combined_df = prepare_plotting_data(data_for_plotting)

# Create the violin plot
plt.figure(figsize=(15, 8))
ax = sns.violinplot(data=combined_df, x='dataset', y='weight', palette='Set2')

# Customize the plot
plt.title('Distribution of Weight Above Weak Layers', fontsize=16, fontweight='bold')
plt.xlabel('Dataset Type', fontsize=14)
plt.ylabel('Weight (kg/m^2)', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)

# Add statistical information as text
# Get clean data for statistics
LOC_clean = LOC_results_df[LOC_results_df['weight_above_layer_of_concern'].notna()]
ECTP_clean = ECTP_results_df[ECTP_results_df['weight_above_ECTP_layer_of_propagation'].notna()]
Crown_clean = pits_on_crowns_with_LOC_results_df[pits_on_crowns_with_LOC_results_df['weight_above_layer_of_concern'].notna()]
Flank_clean = pits_on_flanks_with_LOC_results_df[pits_on_flanks_with_LOC_results_df['weight_above_layer_of_concern'].notna()]

stats_text = f"""
Layer of Concern:
  n = {len(LOC_clean)}
  Mean = {LOC_clean['weight_above_layer_of_concern'].mean():.1f} kg
  Median = {LOC_clean['weight_above_layer_of_concern'].median():.1f} kg

ECTP Propagation:
  n = {len(ECTP_clean)}
  Mean = {ECTP_clean['weight_above_ECTP_layer_of_propagation'].mean():.1f} kg
  Median = {ECTP_clean['weight_above_ECTP_layer_of_propagation'].median():.1f} kg

Crown LOC:
  n = {len(Crown_clean)}
  Mean = {Crown_clean['weight_above_layer_of_concern'].mean():.1f} kg
  Median = {Crown_clean['weight_above_layer_of_concern'].median():.1f} kg

Flank LOC:
  n = {len(Flank_clean)}
  Mean = {Flank_clean['weight_above_layer_of_concern'].mean():.1f} kg
  Median = {Flank_clean['weight_above_layer_of_concern'].median():.1f} kg
"""

plt.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=9, 
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary statistics
print("Summary Statistics:")
print(f"Layer of Concern - Count: {len(LOC_clean)}, Mean: {LOC_clean['weight_above_layer_of_concern'].mean():.1f} kg, Median: {LOC_clean['weight_above_layer_of_concern'].median():.1f} kg")
print(f"ECTP Propagation - Count: {len(ECTP_clean)}, Mean: {ECTP_clean['weight_above_ECTP_layer_of_propagation'].mean():.1f} kg, Median: {ECTP_clean['weight_above_ECTP_layer_of_propagation'].median():.1f} kg")
print(f"Crown LOC - Count: {len(Crown_clean)}, Mean: {Crown_clean['weight_above_layer_of_concern'].mean():.1f} kg, Median: {Crown_clean['weight_above_layer_of_concern'].median():.1f} kg")
print(f"Flank LOC - Count: {len(Flank_clean)}, Mean: {Flank_clean['weight_above_layer_of_concern'].mean():.1f} kg, Median: {Flank_clean['weight_above_layer_of_concern'].median():.1f} kg")


In [ ]:
# Recommended solution for missing density data
def get_density_with_fallback(hand_hardness, grain_form, df=geldsetzer_df):
    """
    Enhanced density lookup with fallback values for missing grain forms.
    
    Parameters:
    hand_hardness (str): Hand hardness value
    grain_form (str): Grain form code
    df (pd.DataFrame): The Geldsetzer table DataFrame
    
    Returns:
    float: Density value, with fallback if not available in table
    """
    try:
        return df.loc[hand_hardness, grain_form]
    except KeyError:
        # Fallback density estimates based on common values for missing grain forms
        fallback_densities = {
            'SH': 180,  # Surface hoar - typically low density
            'MF': 300,  # Melt forms - typically higher density
            'IF': 400,  # Ice formations - high density
            'CR': 250,  # Crusts - medium-high density
            None: 200   # Default fallback for completely unknown
        }
        
        fallback_density = fallback_densities.get(grain_form, 200)
        print(f"Using fallback density {fallback_density} kg/m³ for hardness={hand_hardness}, grain_form={grain_form}")
        return fallback_density

# Example of how to implement the improved function:
# Replace the get_density call in calculate_weight_above_layer with:
# density = get_density_with_fallback(layer.hardness, grain_form)

print("Fallback density function created. To use this:")
print("1. Replace get_density calls with get_density_with_fallback in the weight calculation functions")
print("2. This will reduce NaN values by providing reasonable density estimates")
print("3. Monitor the fallback usage to identify which grain forms need better data")


In [ ]:
# Verify the violin plot KDE issue
print("=== VIOLIN PLOT KDE ISSUE ANALYSIS ===")

# Let's examine the flank data specifically
flank_data = combined_df[combined_df['dataset'] == 'Indicated Layer of Concern (Pits on Flanks)']
print(f"Flank data statistics:")
print(f"  Count: {len(flank_data)}")
print(f"  Min: {flank_data['weight'].min():.2f}")
print(f"  Max: {flank_data['weight'].max():.2f}")
print(f"  Mean: {flank_data['weight'].mean():.2f}")
print(f"  Any negative values: {(flank_data['weight'] < 0).any()}")

# Check all datasets for negative values
print(f"\nAll datasets minimum values:")
for dataset in combined_df['dataset'].unique():
    data_subset = combined_df[combined_df['dataset'] == dataset]
    print(f"  {dataset}: Min = {data_subset['weight'].min():.2f}, Count = {len(data_subset)}")

# Demonstrate KDE effect with a simple example
from scipy import stats
import numpy as np

print(f"\n=== KDE SMOOTHING DEMONSTRATION ===")
# Use flank data for demonstration
flank_weights = flank_data['weight'].values
kde = stats.gaussian_kde(flank_weights)

# Generate points for the KDE curve
x_range = np.linspace(flank_weights.min() - 50, flank_weights.max() + 50, 1000)
kde_values = kde(x_range)

# Find where KDE curve extends below zero
negative_kde_points = x_range[x_range < 0]
if len(negative_kde_points) > 0:
    print(f"KDE curve extends from {x_range.min():.1f} to {x_range.max():.1f}")
    print(f"Even though actual data ranges from {flank_weights.min():.1f} to {flank_weights.max():.1f}")
    print(f"KDE extends {abs(x_range.min()):.1f} units below zero!")


In [ ]:
# CORRECTED VIOLIN PLOT - No extension below zero
print("=== CREATING CORRECTED VIOLIN PLOT ===")

# Create side-by-side violin plots with proper constraints
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Original plot (with the KDE issue)
sns.violinplot(data=combined_df, x='dataset', y='weight', ax=ax1, palette='Set2')
ax1.set_title('Original Violin Plot\n(KDE extends below zero)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Dataset Type', fontsize=12)
ax1.set_ylabel('Weight (kg/m²)', fontsize=12)
ax1.tick_params(axis='x', rotation=45, labelsize=10)
ax1.grid(True, alpha=0.3)

# Corrected plot (constrained to realistic values)
sns.violinplot(data=combined_df, x='dataset', y='weight', ax=ax2, palette='Set2')
ax2.set_title('Corrected Violin Plot\n(Y-axis starts at 0)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Dataset Type', fontsize=12)
ax2.set_ylabel('Weight (kg/m²)', fontsize=12)
ax2.tick_params(axis='x', rotation=45, labelsize=10)
ax2.grid(True, alpha=0.3)

# KEY FIX: Set y-axis to start at 0 (no negative values shown)
ax2.set_ylim(bottom=0)

# Add data points to show actual values
sns.stripplot(data=combined_df, x='dataset', y='weight', ax=ax2, 
              color='black', alpha=0.6, size=3)

plt.tight_layout()
plt.show()

# Alternative solution: Box plots (which don't have the KDE issue)
plt.figure(figsize=(15, 8))
ax3 = sns.boxplot(data=combined_df, x='dataset', y='weight', palette='Set2')
plt.title('Alternative: Box Plot Distribution\n(No KDE artifacts)', fontsize=16, fontweight='bold')
plt.xlabel('Dataset Type', fontsize=14)
plt.ylabel('Weight (kg/m²)', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Violin plot issue resolved!")
print("The 'extension below zero' was a visualization artifact, not a data problem.")


Next Steps
- Figure out why violin plots go below 0
- Think about how to compare / align theoretical benchmark snow profiles w/real world profiles
    - Particularly in crown / flank pits
    - Key attributes of benchmark profiles
        - Flexural rigidity & weight
    - Plot profiles on top of each other
- Add bending stiffness to calculations
    - Calculate stiffness at the same time as weight
- Get SnowPilot density info
- Plot weight vs bending stiffness for each pit (each point on plot represents a "Weak Layer")